# Tests de funcionalidad de la API de series de tiempo

In [114]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

## Variables

In [2]:
BASE_URL = 'http://186.33.215.34/'
METADATA_URL = 'http://infra.datos.gob.ar/catalog/modernizacion/dataset/1/distribution/1.2/download/series-tiempo-metadatos.csv'
ENDPOINT_URL = BASE_URL + 'series/api/series/'

In [3]:
series_metadata = pd.read_csv(METADATA_URL)
series_metadata.tail(5)

,catalogo_id,dataset_id,distribucion_id,serie_id,indice_tiempo_frecuencia,serie_titulo,serie_unidades,serie_descripcion,distribucion_titulo,distribucion_descripcion,...,dataset_descripcion,dataset_tema,serie_indice_inicio,serie_indice_final,serie_valores_cant,serie_dias_no_cubiertos,serie_actualizada,serie_valor_ultimo,serie_valor_anterior,serie_var_pct_anterior
20190,sspm,99,99.3,99.3_ING_2008_0_17,R/P1M,ipc_nivel_general,Índice abr-2008=100,Índice de precios al consumidor nivel general....,"Índice de Precios al Consumidor, por grupos. D...",Índice de Precios al Consumidor. Apertura por ...,...,Índice de Precios al Consumidor. Apertura por ...,Precios,2006-12-01,2013-12-01,85,1630,False,166.840000,164.510000,0.014163
20191,sspm,99,99.3,99.3_IR_2008_0_13,R/P1M,ipc_regulados,Índice abr-2008=100,Índice de precios al consumidor regulados. Val...,"Índice de Precios al Consumidor, por grupos. D...",Índice de Precios al Consumidor. Apertura por ...,...,Índice de Precios al Consumidor. Apertura por ...,Precios,2006-12-01,2013-12-01,85,1630,False,161.040000,159.290000,0.010986
20192,sspm,99,99.3,99.3_IR_2008_0_9,R/P1M,ipc_resto,Índice abr-2008=100,Índice de precios al consumidor IPC resto. Val...,"Índice de Precios al Consumidor, por grupos. D...",Índice de Precios al Consumidor. Apertura por ...,...,Índice de Precios al Consumidor. Apertura por ...,Precios,2006-12-01,2013-12-01,85,1630,False,161.740000,159.250000,0.015636
20193,sspm,99,99.3,99.3_PREIR_2008_0_40,R/P1M,precios_relativos_estacionales_ipc_resto,Índice abr-2008=100,Índice de precios al consumidor precios relati...,"Índice de Precios al Consumidor, por grupos. D...",Índice de Precios al Consumidor. Apertura por ...,...,Índice de Precios al Consumidor. Apertura por ...,Precios,2006-12-01,2013-12-01,85,1630,False,137.591196,138.411303,-0.005925
20194,sspm,99,99.3,99.3_PRRIR_2008_0_37,R/P1M,precios_relativos_regulados_ipc_resto,Índice abr-2008=100,Índice de precios al consumidor precios relati...,"Índice de Precios al Consumidor, por grupos. D...",Índice de Precios al Consumidor. Apertura por ...,...,Índice de Precios al Consumidor. Apertura por ...,Precios,2006-12-01,2013-12-01,85,1630,False,99.567207,100.025118,-0.004578


In [4]:
series_metadata.serie_id.count()

20195

## Chequeo de todas las series 

Le pegamos al endpoint para todas las series y verificamos que la API devuelve una respuesta satisfactoria (status code 200). Contamos la cantidad de casos satisfactorios (True) y no (False)

In [5]:
def api_series_head(serie_id):
    return requests.head(ENDPOINT_URL, params={'ids': serie_id}).status_code == 200

### Cantidad de respuestas válidas

In [7]:
valid_responses = series_metadata.serie_id[:10].apply(api_series_head)
valid_responses.value_counts()

True    10
Name: serie_id, dtype: int64

### En porcentajes

In [9]:
valid_responses.value_counts().apply(lambda x: x/len(valid_responses))

True    1.0
Name: serie_id, dtype: float64

## Chequeo de modos de representación

In [156]:
def api_call(serie, limit=1000, **kwargs):
    call_params = {'ids': serie, 'format': 'csv', 'limit': limit}
    call_params.update(kwargs)
    csv = StringIO(requests.get(ENDPOINT_URL, params=call_params).content.decode('utf8'))
    api_csv = pd.read_csv(csv)

    return api_csv

def get_source_csv(serie):
    response = requests.get(ENDPOINT_URL, params={'ids': serie, 'metadata': 'only'}).json()
    distribution_url = response['meta'][1]['distribution']['downloadURL']
    title = response['meta'][1]['field']['title']
    orig_csv = pd.read_csv(distribution_url)[['indice_tiempo', title]]

    return orig_csv, title

In [203]:
serie = series_metadata.serie_id.sample(1).values[0]
serie

'142.2_CREDI_2001_T_24'

### Serie original

In [164]:
orig_csv, title = get_source_csv(serie)

### Serie de la API: valor original

In [165]:
api_csv = api_call(serie)

api_csv.head(1)

,indice_tiempo,nf18_yield
0,2013-01-02,0.102487


In [167]:
orig_csv = orig_csv[:len(api_csv)]
equality_check = np.isclose(orig_csv[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    1000
dtype: int64

### Cambio absoluto

In [209]:
api_csv = api_call(serie, representation_mode='change')

In [210]:
orig_csv_change, title = get_source_csv(serie)
orig_csv_change[title] = orig_csv_change[title].diff(1)
orig_csv_change = orig_csv_change[:len(api_csv)]

In [211]:
equality_check = np.isclose(orig_csv_change[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    84
dtype: int64

### Cambio porcentual

In [206]:
api_csv = api_call(serie, representation_mode='percent_change')

In [207]:
orig_csv_pct_change, title = get_source_csv(serie)
orig_csv_pct_change[title] = orig_csv_pct_change[title].pct_change()
orig_csv_pct_change = orig_csv_pct_change[:len(api_csv)]

In [208]:
equality_check = np.isclose(orig_csv_pct_change[title], api_csv[title], equal_nan=True)
equal_df = pd.Series(equality_check)

equal_df.value_counts()

True    84
dtype: int64